In [3]:
## @file imports.py
#  @brief Import necessary libraries for video processing and visualization.
#  @details This script uses OpenCV for video manipulation, NumPy for numerical operations, 
#  Matplotlib for frame visualization, and IPython for interactive display updates.

# @brief Import OpenCV library for video and image processing.
import cv2  

# @brief Import NumPy library for numerical computations and array manipulations.
import numpy as np  

# @brief Import Matplotlib library for displaying frames as images.
import matplotlib.pyplot as plt  

# @brief Import clear_output from IPython to update displayed output interactively.
from IPython.display import clear_output  


In [ ]:
##
# @brief Dummy callback function for trackbars
# @param x Trackbar position value (unused)
def nothing(x):
    pass

##
# @brief Read input image from file
# @note Image should be in the same directory as the script
image = cv2.imread('image.png')

##
# @brief Create named window for HSV trackbars
cv2.namedWindow('HSV Trackbars')

##
# @brief Create trackbars for adjusting HSV ranges
# @note H (Hue) ranges from 0-179 in OpenCV
# @note S (Saturation) and V (Value) range from 0-255
cv2.createTrackbar('H_min', 'HSV Trackbars', 0, 179, nothing)    # Minimum Hue
cv2.createTrackbar('H_max', 'HSV Trackbars', 179, 179, nothing)  # Maximum Hue
cv2.createTrackbar('S_min', 'HSV Trackbars', 0, 255, nothing)    # Minimum Saturation
cv2.createTrackbar('S_max', 'HSV Trackbars', 255, 255, nothing)  # Maximum Saturation
cv2.createTrackbar('V_min', 'HSV Trackbars', 0, 255, nothing)    # Minimum Value
cv2.createTrackbar('V_max', 'HSV Trackbars', 255, 255, nothing)  # Maximum Value

##
# @brief Main processing loop
# @details Continuously updates mask based on trackbar positions until 'q' is pressed
while True:
    ##
    # @brief Convert BGR image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    ##
    # @brief Get current positions of all trackbars
    h_min = cv2.getTrackbarPos('H_min', 'HSV Trackbars')
    h_max = cv2.getTrackbarPos('H_max', 'HSV Trackbars')
    s_min = cv2.getTrackbarPos('S_min', 'HSV Trackbars')
    s_max = cv2.getTrackbarPos('S_max', 'HSV Trackbars')
    v_min = cv2.getTrackbarPos('V_min', 'HSV Trackbars')
    v_max = cv2.getTrackbarPos('V_max', 'HSV Trackbars')
    
    ##
    # @brief Create arrays for lower and upper HSV bounds
    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    
    ##
    # @brief Create binary mask and apply it to original image
    # @details Pixels within HSV range become white (255), others become black (0)
    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(image, image, mask=mask)
    
    ##
    # @brief Display original image, mask, and masked result
    cv2.imshow('Original', image)
    cv2.imshow('Mask', mask)
    cv2.imshow('Result', result)
    
    ##
    # @brief Print current HSV range values
    # @note \r is used for inline updating of values
    print(f"\rHSV Range - H: [{h_min}, {h_max}], S: [{s_min}, {s_max}], V: [{v_min}, {v_max}]", end='')
    
    ##
    # @brief Check for 'q' key press to exit loop
    # @return Prints final HSV ranges before exiting
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("\nFinal HSV Range:")
        print(f"lower = np.array([{h_min}, {s_min}, {v_min}])")
        print(f"upper = np.array([{h_max}, {s_max}, {v_max}])")
        break

##
# @brief Clean up and close all OpenCV windows
cv2.destroyAllWindows()

HSV Range - H: [0, 179], S: [0, 255], V: [0, 255]

KeyboardInterrupt: 

In [ ]:
## @file path_tracking.py
#  @brief A program to track and highlight a specific color range in a video, save the processed video, and visualize it frame by frame.
#  @details This program uses OpenCV to process video frames, apply a color filter, identify contours, and highlight the largest contour's centroid. 
#  It then saves the processed video to an output file.

videoFrames = []  # List to store processed video frames

## @brief Displays a single video frame using matplotlib.
#  @param frame The video frame to be displayed.
def display_frame(frame):
    # Convert the frame from BGR to RGB for matplotlib visualization
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(rgb_frame)
    plt.axis("off")  # Hide axes
    plt.show()
    clear_output(wait=True)  # Clear previous frame for smooth updating

## @brief Main function to process the video, track a color range, and save the output.
def main():
    # Input video file
    video = "raw_video_feed.mp4"
    capture = cv2.VideoCapture(video)  # Open video capture object

    # Define HSV color range for filtering
    lower = np.array([88, 80, 0])  # Lower bound of HSV color range
    upper = np.array([110, 160, 101])  # Upper bound of HSV color range

    kernel = np.ones((5, 5), np.uint8)  # Kernel for morphological operations

    while True:
        ret, frame = capture.read()  # Read a video frame
        if not ret:
            break  # Exit loop if no frame is returned (end of video)

        # Convert frame to HSV color space
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Create a binary mask for the specified color range
        mask = cv2.inRange(hsv, lower, upper)

        # Display the mask in a separate window (debugging purposes)
        cv2.imshow("Mask", mask)
        key = cv2.waitKey(1) & 0xFF  # Wait for a key press
        if key == ord('q'):  # Exit loop if 'q' is pressed
            break

        # Find contours in the binary mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours) > 0:
            # Identify the largest contour
            largest_contour = max(contours, key=cv2.contourArea)

            # Compute centroid of the largest contour
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
            else:
                cx, cy = 0, 0  # Handle division by zero case

        # Convert frame to grayscale and create a two-tone composite image
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_bgr = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
        blue_part = cv2.bitwise_and(frame, frame, mask=mask)
        mask_inv = cv2.bitwise_not(mask)
        gray_part = cv2.bitwise_and(gray_bgr, gray_bgr, mask=mask_inv)
        final_frame = cv2.add(blue_part, gray_part)

        # Draw a circle at the centroid of the largest contour
        if cx >= 0 and cy >= 0:
            cv2.circle(final_frame, (cx, cy), 20, (0, 0, 255), -1)

        # Append the processed frame to the list
        videoFrames.append(final_frame)
        display_frame(final_frame)  # Display the processed frame

    capture.release()  # Release video capture object

main()

# @brief Save the processed video frames as a new video file.
frame_height, frame_width = videoFrames[0].shape[:2]  # Get frame dimensions
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Define codec for video writing
out = cv2.VideoWriter('output_video.mp4', fourcc, 30, (frame_width, frame_height))  # Create video writer object

# Write all frames to the output video
for frame in videoFrames:
    out.write(frame)

out.release()  # Release video writer object
print("Video saved as 'output_video.mp4'")



KeyboardInterrupt: 